In [1]:
import pathpy as pp
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import numpy as np
from datetime import timedelta, datetime
from ast import literal_eval
import networkx as nx
import folium
import time
import warnings
import pickle
import sys

warnings.filterwarnings('ignore')
print("Geopandas has version {}".format(gpd.__version__))
print("Movingpandas has version {}".format(mpd.__version__))

# add paths for modules
sys.path.append('../visualization')
sys.path.append('../features')
# import modules
import visualize
import geometry_utils
from maritime_traffic_network import MaritimeTrafficNetwork
from MOGen_path_prediction import MOGenPathPrediction
import prediction_model_evaluation_metrics as metrics

ModuleNotFoundError: No module named 'pathpy'

In [ ]:
# Load a model from pickle
datasize = 'full'
location = 'tromso'
model_date = '202204'
train_date = '202205'
test_date = '202206'
DP_tol = 10
min_samples = 13

model_name = model_date+'_waypoints_DP'+str(DP_tol)+'_HDBSCAN'+str(min_samples)+'_'+location+'_'+datasize+'_UTM'
model_path = '../../models/networks/best_networks/' + model_name + '.obj'
fileObj = open(model_path, 'rb')
network = pickle.load(fileObj)
fileObj.close()
network.hyperparameters

In [ ]:
# Load training data from file
filename = model_name+'_'+train_date+'_paths.csv'
training_data = pd.read_csv('../../data/paths/'+filename)
training_data['path'] = training_data['path'].apply(literal_eval)
training_data = gpd.GeoDataFrame(training_data, geometry=gpd.GeoSeries.from_wkt(training_data['geometry']), crs=network.crs)
training_data = training_data[training_data['message']=='success']
# extract paths from the training data
training_paths = training_data['path'].tolist()
training_data.info()

In [ ]:
# Load test data from file
filename = model_name+'_'+test_date+'_paths.csv'
test_data = pd.read_csv('../../data/paths/'+filename)
test_data['path'] = test_data['path'].apply(literal_eval)
test_data = gpd.GeoDataFrame(test_data, geometry=gpd.GeoSeries.from_wkt(test_data['geometry']), crs=network.crs)
test_data = test_data[test_data['message']=='success']

In [ ]:
# Load respective trajectories for evaluation
traj_file = test_date+'_points_'+location+'_cleaned_meta_full_dualSplit_2'
filename = '../../data/processed/' + traj_file + '.parquet'
traj_gdf = gpd.read_parquet(filename)
crs = network.crs  # Coordinate reference system
traj_gdf.to_crs(crs, inplace=True)  # Transformation
test_trajectories = mpd.TrajectoryCollection(traj_gdf, traj_id_col='mmsi', obj_id_col='mmsi')

In [ ]:
# Train MOGen Model
mogen = MOGenPathPrediction()
mogen.train(training_paths, max_order=3, model_selection=True)

In [ ]:
mogen.model.plot()

In [ ]:
# make predictions for a start node
#predictions = mogen.predict_paths(start_node=[130], n_paths=100)

In [ ]:
example = test_data.iloc[10]
mmsi = example.mmsi
start_node = [example.path[0]]
end_node = example.path[-1]
print(start_node)
true_path = example.path
trajectory = test_trajectories.get_trajectory(mmsi)
#start_node = [130]
#predictions = mogen.predict_next_nodes(start_node=start_node, G=network.G, n_predictions=5, n_steps=6, n_walks=200)
predictions = mogen.predict_path(start_node, end_node, network.G, n_predictions=5, n_walks=2000, verbose=True)

In [ ]:
selection = range(0, 6, 2)  # len(test_data)
test_paths = test_data.iloc[selection]
eval_results, fig = metrics.evaluate(mogen, 'path', test_paths, test_trajectories, network, MOGen_n_walks=1000, eval_mode='path')

In [ ]:
nan_mask =eval_results.isna().any(axis=1)
eval_results[nan_mask]

In [ ]:
map = visualize.map_prediction_and_ground_truth(predictions, start_node, trajectory, true_path, network, min_passages=1, opacity=0.2, location=location, end_node=end_node)
map.save('../../reports/maps/'+model_name+'_MOGen_predictions.html')
print(f'Map saved under {"../../reports/maps/"+model_name+"_MOGen_predictions.html"}')